# Création d'une segmentation client pour Olist

-------------------------------------------------------------------------------------------------------------------------------

Problématique : identifier différents comportements des clients afin de pouvoir faire du ciblage marketing et augmenter ainsi la rentabilité de l'entreprise

Nous disposons de plusieurs informations anonymisées de la société Olist sur : 
- les commandes
- les clients
- les produits
- les catégories des produits
- des informations sur le paiment des commandes
- les vendeurs.

Il est important de noter qu'une commande peut contenir des produits de différents vendeurs.


In [ ]:
#notre package de fonctionnalités
from Package import Scripts_Analyse01 as pk

import pandas as pd
import numpy as np
import missingno
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats
from scipy.stats import pearsonr
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from yellowbrick.features import ParallelCoordinates
from plotly.graph_objects import Layout
import jenkspy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.features import PCA as PCA_yellow
from yellowbrick.style import set_palette
# Import train_test_split function
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, f_classif,mutual_info_classif
from skfeature.function.similarity_based import fisher_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
 
import mlflow
from datetime import datetime

Regardons les différentes tables à notre disposition et s'il y a des doublons où des valeurs manquantes

Commençons par la table des commandes.

In [ ]:
orders = pd.read_csv('./input/olist_orders_dataset.csv')

In [ ]:
orders.head(5)

In [ ]:
orders.shape

Nous avons 99 441 lignes et 8 variables.

In [ ]:
orders[['order_id']].duplicated().sum()

Nous avons une ligne par commande et il n'y a pas de doublons dans cette table.

Regardons si cette table contient des données manquantes.

In [ ]:
tab=pk.del_Nan(orders, 0.0000000001,0, 0)

In [ ]:
tab

Cette table contient très peu de données manquantes.

In [ ]:
pk.matrix_vm(orders, (16,8), (0.60, 0.64, 0.49))

Nous observons des données manquantes pour les dates de livraison par exemple. Nous pouvons conserver ces données car ce sont surement des commandes non livrées.

In [ ]:
orders.loc[pd.isna(orders["order_approved_at"])==True]

Nous avons des données manquantes pour les commandes annulées, ce qui est normal car nous n'avons pas les informations.
Est ce qu'il est intéressant de conserver ces données ??????

Il serait interessant de créer des variables comme le délai de livraison au client, délai de livraison au transporteur et le délai entre la livraison transporteur et la livraison client.

Nous créerons une fonction pour la création des variables et l'aggrégation ensuite. Passons à l'étude de la table reviews.

In [ ]:
reviews = pd.read_csv('./input/olist_order_reviews_dataset.csv', nrows = num_rows)

In [ ]:
reviews.head(5)

In [ ]:
reviews.shape

In [ ]:
reviews[['review_id', 'order_id']].duplicated().sum()

In [ ]:
tab=pk.del_Nan(reviews, 0.0000000001,0, 0)

In [ ]:
tab

Il n'y a pas de doublons dans cette table. Les colonnes review_comment_title et review_comment_message	contiennent des données vides, mais le client n'est pas obligé de remplir ces données. Nous créerons des colonnes binaires sur ces colonnes et nous ferons des aggrégations par commande par la suite.
Etudions la table concernant les paiements.

In [ ]:
payments = pd.read_csv('./input/olist_order_payments_dataset.csv', nrows = num_rows)

In [ ]:
payments.head(5)

In [ ]:
payments[['order_id', 'payment_sequential']].duplicated().sum()

In [ ]:
tab=pk.del_Nan(payments, 0.0000000001,0, 0)
tab

Il n'y a pas de doublons et il n'y a pas de données manquantes dans cette table. Nous allons calculer les sommes par commandes et les moyennes. Nous créerons une fonction pour faire ce traitement. Analysons la table des produits, des catégories de produits et des vendeurs.

In [ ]:
prod_transl = pd.read_csv('./input/product_category_name_translation.csv', nrows = num_rows)

In [ ]:
products = pd.read_csv('./input/olist_products_dataset.csv', nrows = num_rows)

In [ ]:
sellers = pd.read_csv('./input/olist_sellers_dataset.csv', nrows = num_rows)

In [ ]:
prod_transl.head(5)

In [ ]:
prod_transl[['product_category_name']].duplicated().sum()

In [ ]:
tab=pk.del_Nan(prod_transl, 0.0000000001,0, 0)
tab

In [ ]:
products.head(5)

In [ ]:
products[['product_id']].duplicated().sum()

In [ ]:
tab=pk.del_Nan(products, 0.0000000001,0, 0)
tab

Nous pouvons remplacer les vides par 0 pour la table produit, car cela nous indiquera qu'il n'y a pas d'information pour ces produits.

In [ ]:
sellers.head(5)

In [ ]:
sellers[['seller_id']].duplicated().sum()

In [ ]:
tab=pk.del_Nan(sellers, 0.0000000001,0, 0)
tab

Il n'y a pas de doublons dans ces tables. Il existe des données manquantes dans la tables produits, mais très peu. Nous conservons les données telles quelles. Regardons la table contenant les produits par commande.

In [ ]:
items = pd.read_csv('./input/olist_order_items_dataset.csv', nrows = num_rows)

In [ ]:
items.head(5)

In [ ]:
items[['order_id','product_id', 'order_item_id']].duplicated().sum()

In [ ]:
tab=pk.del_Nan(items, 0.0000000001,0, 0)
tab

Il n'y a aucun doublons et aucunes données manquantes.

Regardons la table clients

In [ ]:
customer_obj = pd.read_csv('./input/olist_customers_dataset.csv', nrows = num_rows)

In [ ]:
customer_obj

Nous observons qu'un customer_unique_id peut avoir plusieurs customer_id. Un client peut donc avoir plusieurs comptes client. Il faudra réaliser une aggrégation grâce à l'id unique du client.

In [ ]:
customer_obj[['customer_id','customer_unique_id']].duplicated().sum()

In [ ]:
tab=pk.del_Nan(customer_obj, 0.0000000001,0, 0)
tab

Il n'y a pas de doublons et il n'y a pas de données manquantes dans cette table.

Regardons la table de geolocalisation.

In [ ]:
geoloc = pd.read_csv('./input/olist_geolocation_dataset.csv', nrows = num_rows)   

In [ ]:
geoloc.head(5)

In [ ]:
geoloc[['geolocation_zip_code_prefix','geolocation_city','geolocation_state']].duplicated().sum()

In [ ]:
geoloc.shape

Nous avons beaucoup de doublons dans cette table. Nous n'allons pas l'utiliser.
Nous pourrions par la suite rechercher une table des localisations des villes du Brésil sur internet afin d'ajouter une localisation des clients par ville.

Nous pouvons à présent fusionner nos tables et faire des aggrégations par client unique.

In [ ]:

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# One-hot encoding for categorical columns with get_dummies
#####encoder les colonnes catégorielles
def one_hot_encoder(df, exclude=[], nan_as_category = True):
    cols=filter(lambda x: ("_id" not in (x)) and x not in (exclude), df.columns)
    original_columns = list(cols)
    categorical_columns = [col for col in original_columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

def agg_by_group(df_test2, tab, cols, agg_name, col_index, col_by_cat):
    for col in cols:
        df_calcul=df_test2.pivot_table(values=col, index=col_index, columns=col_by_cat, aggfunc=agg_name).reset_index()
        lt_col=list(filter(lambda x: x not in (col_index), df_calcul.columns))
        lt_col=[e + "_"+str(col) for e in lt_col]
        lt_col.insert(0,col_index)
        df_calcul.columns=pd.Index([e for e in lt_col])
        df_calcul=df_calcul.fillna(0)
        tab=tab.merge(df_calcul, how='left', on=col_index)
        #display(tab)
    return tab

def difference_dates(date2, date1, new_column, df, var_i=0):    
    df2=df.copy()
    if var_i==0:
        df2[new_column] = (pd.to_datetime(df2[date2]) - pd.to_datetime(df2[date1])).dt.days
    if var_i==1:
        df2[new_column] = (pd.to_datetime(date2) - pd.to_datetime(df2[date1])).dt.days
    return df2

# Preprocess application_train.csv and application_test.csv
def orders(num_rows = None, nan_as_category = False):
    # Read data and merge
    ####lecture des fichiers application_train et application test
    df = pd.read_csv('./input/olist_orders_dataset.csv', nrows= num_rows)
    print("df shape: {}".format(len(df)))
    
    #calcul aggregation - add column
    df=difference_dates("order_delivered_customer_date", "order_approved_at", 'time_delivered_approved_customer', df)
    df=difference_dates("order_approved_at", "order_purchase_timestamp", 'time_approval_purchase', df)
    df=difference_dates("order_delivered_carrier_date", "order_approved_at", 'time_delivery_carrier_approved', df)
    df=difference_dates("order_delivered_customer_date", "order_delivered_carrier_date", 'time_delivery_customer_carrier', df)
    df=difference_dates("order_delivered_customer_date", "order_estimated_delivery_date", 'time_delivery_customer_estimated', df)
        
    df["order_purchase_year_month"]=pd.to_datetime(df["order_purchase_timestamp"]).dt.strftime('%Y%m')
    df["order_purchase_timestamp"]=pd.to_datetime(df["order_purchase_timestamp"])
    df=df.drop(['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'], axis=1)
    data, data_cat = one_hot_encoder(df, ['order_purchase_timestamp', "order_id"])
    del df
    gc.collect()
    return data, data_cat


# Preprocess bureau.csv and bureau_balance.csv
def order_reviews(num_rows = None, nan_as_category = True):
    date1 = datetime.now()
    ####lecture des data et encodage des variable cat
    reviews = pd.read_csv('./input/olist_order_reviews_dataset.csv', nrows = num_rows)
    #bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    print("reviews shape: {}".format(len(reviews)))
    ####calcul d'agregat pour les variables

    reviews['review_comment_message_ind']=[0 if pd.isna(i.review_comment_message)==True else 1 for i in reviews.itertuples()]
    reviews['review_comment_title_ind']=[0 if pd.isna(i.review_comment_title)==True else 1 for i in reviews.itertuples()]
    reviews['review_created_days'] = [None if pd.isna(i.review_creation_date)==True else (pd.to_datetime(i.review_creation_date) - date1).days for i in reviews.itertuples()]
    reviews['review_answer_days'] = [None if pd.isna(i.review_answer_timestamp)==True else (pd.to_datetime(i.review_answer_timestamp) - date1).days for i in reviews.itertuples()]
    reviews_aggregations = {
                            'review_score': ['mean'],
                            'review_comment_message_ind': ['sum'],
                            'review_comment_title_ind': ['sum'],
                            'review_created_days': ['sum'],
                            'review_answer_days': ['sum']
                           }
    
    del reviews["review_comment_message"]
    del reviews["review_comment_title"]
    del reviews["review_creation_date"]
    del reviews["review_answer_timestamp"]
    reviews_agg = reviews.groupby('order_id').agg(reviews_aggregations)
    reviews_agg.columns = pd.Index([e[0] + "_" + e[1].lower() for e in reviews_agg.columns.tolist()])
    del reviews
    print("reviews agg shape: {}".format(len(reviews_agg)))
    gc.collect()
    return reviews_agg

# Preprocess previous_applications.csv
def order_payments(num_rows = None, nan_as_category = True):
    payments = pd.read_csv('./input/olist_order_payments_dataset.csv', nrows = num_rows)
    #Encode categoricals variables
    data, data_cat = one_hot_encoder(payments)
    #create aggregation on quantitatives variables
    paid_aggregations = {'payment_value': ['sum'],
                  'payment_installments': ['sum']}
    for col in data_cat:
        if col!="order_id":
            paid_aggregations[col] = ['sum']
    paid_agg = data.groupby("order_id").agg(paid_aggregations)
    #rename columns
    paid_agg.columns = pd.Index([e[0] + "_" + e[1].lower() for e in paid_agg.columns.tolist()])
    
    #aggregation quantitatives var by group
    payment_aggregations=agg_by_group(payments, paid_agg, ["payment_value"], "sum","order_id","payment_type" )
    del payments,data_cat
    gc.collect()
    return payment_aggregations

def product_category_translation(num_rows = None, nan_as_category = True):
    prod_transl = pd.read_csv('./input/product_category_name_translation.csv', nrows = num_rows)
    #ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    
    gc.collect()
    return prod_transl
    
# Preprocess installments_payments.csv
def products(num_rows = None, nan_as_category = True):
    products = pd.read_csv('./input/olist_products_dataset.csv', nrows = num_rows)
    cat = product_category_translation(num_rows,nan_as_category)
    #ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    products=products.merge(cat, how="left", on="product_category_name")
    del products["product_category_name"]
    data, data_cat = one_hot_encoder(products)
    data=data.fillna(0)
    del cat
    gc.collect()
    return data, data_cat

# Preprocess credit_card_balance.csv
def sellers(num_rows = None, nan_as_category = True):
    sellers = pd.read_csv('./input/olist_sellers_dataset.csv', nrows = num_rows)
    #cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    
    #del cc
    gc.collect()
    return sellers


# Preprocess POS_CASH_balance.csv
def order_items(num_rows = None, nan_as_category = True):
    items = pd.read_csv('./input/olist_order_items_dataset.csv', nrows = num_rows)
    products_obj, prods_cat=products(num_rows = None, nan_as_category = True)
    #pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    items=items.merge(products_obj, how="left", on="product_id")
    #sell=sellers(num_rows = None, nan_as_category = True)
    #items=items.merge(sell, how="left", on="seller_id")
    items_aggregations = {}
    df_num=items.select_dtypes(include=[np.float])
    for col in prods_cat:
        if ("_id" not in col) and ("_date" not in col):
              items_aggregations[col] = ['sum']
        elif col in ["product_id", "seller_id", "item_id"]:
              items_aggregations[col] = ['size']
        elif "_date" in col:
              items_aggregations[col] = ['max']
    
    for col_num in df_num:
        items_aggregations[col_num] = ['mean']
    items_agg = items.groupby("order_id").agg(items_aggregations)
    items_agg.columns = pd.Index([e[0] + "_" + e[1] for e in items_agg.columns.tolist()])
    del items, products_obj
    gc.collect()
    
    return items_agg


def orders_informations(num_rows = None, nan_as_category = True):
    with timer("Orders "):
        df, data_cat = orders(num_rows)
        print("Order df shape:", df.shape)
        gc.collect()
    with timer("Orders reviews"):
        reviews = order_reviews(num_rows)
        print("Reviews df shape:", reviews.shape)
        df = df.merge(reviews, how='left', on='order_id')
        del reviews
        gc.collect()
    with timer("Process order payments"):
        payments = order_payments(num_rows)
        print("Order payments df shape:", payments.shape)
        df = df.merge(payments, how='left', on='order_id')
        del payments
        gc.collect()
    
    with timer("Process order items"):
        items = order_items(num_rows)
        print("Order items df shape:", items.shape)
        df = df.merge(items, how='left', on='order_id')
        del items
        gc.collect()  
    return df


# Preprocess credit_card_balance.csv
def customer(num_rows = None, nan_as_category = True):
    customer_obj = pd.read_csv('./input/olist_customers_dataset.csv', nrows = num_rows)
    #del cc
    gc.collect()
    return customer_obj

# Il y a des doublons dans cette table > une ville ou un code zip a différentes coordonnées
def geolocation(num_rows = None, nan_as_category = True):
    geoloc = pd.read_csv('./input/olist_geolocation_dataset.csv', nrows = num_rows)    
    #del cc
    gc.collect()
    return geoloc

In [ ]:
# lance chaque fonction avec un timer
def main(debug = True):
    num_rows = 1000 if debug else None
    
    ####import fichiers + calcul agregats (new features) et jointure de toutes les tables
    ### + nettoyage memoire
    with timer("Orders informations"):
        df = orders_informations(num_rows)
        print("orders informations shape:", df.shape)
        gc.collect()
    
    with timer("Process customer"):
        customer_obj = customer(num_rows)
        customer_obj = customer_obj.merge(df, how='left', on='customer_id')
        print("Customer df shape:", customer_obj.shape)
        del df
        gc.collect()
        customer_obj.columns = customer_obj.columns.str.strip()    
        customer_obj.columns = customer_obj.columns.str.replace(' ', '_')
        customer_obj.columns = customer_obj.columns.str.replace(r"[^a-zA-Z\d\_]+", "")  
        customer_obj.columns = customer_obj.columns.str.replace(r"[^a-zA-Z\d\_]+", "")
        
        
        ##############a modifier
        ##Moyenne - median by customer
        #count nb commande + nb produit mean + nb prod by category en moy par commande + time moyen
        customer_aggregations={}
        cols=list(customer_obj.columns)
        cols_float=customer_obj.select_dtypes(include=[np.float])
        col_ok=[]       
        
        
        for col in cols:
            if (("_sum" in col) and ("payment_" not in col)) or ('order_status' in col) or ('order_purchase' in col) :
                customer_aggregations[col] = ['sum']
                col_ok.append(col)
            if ("payment_" in col) or "_size" in col:
                customer_aggregations[col] = ['sum', 'mean']
                col_ok.append(col)
            if "order_id" in col or "customer_id" in col:
                customer_aggregations[col] = ["size"]
                col_ok.append(col)
            if "time_" in col:
                customer_aggregations[col] = ['mean']
                col_ok.append(col)
            if 'order_purchase_timestamp' in col:
                customer_aggregations[col] = ['max']
                col_ok.append(col)
        for col in cols_float:
            if col not in col_ok :
                customer_aggregations[col] = ['mean', 'size']
        customer_agg = customer_obj.groupby(["customer_unique_id"]).agg(customer_aggregations)
        customer_agg.columns = pd.Index([e[0] + "_" + e[1].lower() for e in customer_agg.columns.tolist()])
        
        date_max_purchase=customer_agg['order_purchase_timestamp_max'].max()
        customer_agg=difference_dates(date_max_purchase, "order_purchase_timestamp_max", "order_purchase_timestamp_recency", customer_agg,1)
        
        del customer_obj
        gc.collect()
        return customer_agg.reset_index()
    
if __name__ == "__main__":
    #submission_file_name = "submission_kernel02.csv"
    with timer("Full complete run"):
        df = main(False)
        #, feat_importance
       

In [ ]:
df

In [ ]:
df[['customer_unique_id']].duplicated().sum()

In [ ]:
df.shape

Nous avons à présent une ligne par client, au total 96 096 et 169 variables.

Supprimons les colonnes avec des valeurs uniques.

In [ ]:
pk.data_uniqueone_string(df)

In [ ]:
df.shape

Il reste 163 variables

In [ ]:
for i in df.columns:
    print(i)

Nous avions quelques données manquantes. Regardons les de plus près.

In [ ]:
pk.matrix_vm(df, (16,8), (0.60, 0.64, 0.49))

In [ ]:
tab=pk.del_Nan(df, 0.0000000001,0, 0)
tab

In [ ]:
df.loc[pd.isna(df["product_id_size_mean"])==True]

In [ ]:
for i in df.columns:
    if "product_" in i:
        df[i]=df[i].fillna(0)

Regardons les autres colonnes, il y a des valeurs manquantes car la commande n'a pas été approuvée. Remplaçons toutes les valeurs par 0 pour les variables : payment price, freight.

In [ ]:
tab=pk.del_Nan(df, 0.0000000001,0, 0)
tab

In [ ]:
for i in df.columns:
    if ("payment_" in i) or ("price_" in i) or ("freight_" in i)  or ("review_" in i):
        df[i]=df[i].fillna(0)

In [ ]:
tab=pk.del_Nan(df, 0.0000000001,0, 0)
tab

In [ ]:
df["time_delivery_customer_estimated_mean"].max()

Nous décidons de mettre 365 pour les variables time.

In [ ]:
for i in df.columns:
    if ("time_" in i):
        df[i]=df[i].fillna(365)

In [ ]:
tab=pk.del_Nan(df, 0.0000000001,0, 0)
tab

In [ ]:
df

## Analyse exploratoire

In [ ]:
df.describe()

Les colonnes customer_id_size et order_id_size sont identiques. Nous supprimons la colonne customer_id_size

In [ ]:
del df["customer_id_size"]

Il ne semble pas qu'il y ait des données aberrantes.
Cependant, nous observons des variables avec des données uniques. Supprimons ces variables.

In [ ]:
df.shape

Il reste 163 variables

Les données sont peu dispersées au sein des variables, nous observons que toutes les boites à moustache sont aplaties.
Nous observons quand même des valeurs assez hautes. Il serait intéressant de créer des regoupements ordinaux

Realisons un test de skewness pour voir comment se comporte les distributions.

Nous rappelons :
- Si y1=0 alors la distribution est symétrique.
- Si y1>0 alors la distribution est étalée à droite.
- Si y1<0 alors la distribution est étalée à gauche.

In [ ]:
for col in df.select_dtypes(include=['float64']).columns:
    df_i=df[col].loc[pd.isna(df[col])==False]
    print("Variable : "+col+" ----- y1="+ str(df[col].skew()))

La plupart des distributions des variables sont étalées à droite.

Regardons comment nos variables se comportent par rapport à la loi normale grâce au test de kurtosis.

Nous rappelons les informations suivantes :
- Si γ2=0 , alors la distribution a le même aplatissement que la distribution normale.
- Si γ2>0 , alors elle est moins aplatie que la distribution normale : les observations sont plus concentrées.
- Si γ2<0 , alors les observations sont moins concentrées : la distribution est plus aplatie.

In [ ]:
for col in df.select_dtypes(include=['float64']).columns:
    df_i=df[col].loc[pd.isna(df[col])==False]
    print("Variable : "+col+" -- y2="+ str(df[col].kurtosis()))

La majorité des distributions des variables sont moins aplaties que la distribution de la loi normale.

In [ ]:
for col in df.columns:
    if df[col].nunique()<20 and df[col].nunique()>=6:
        pk.graph_barplot(df[col], "Répartition des clients selon la variable"+col, 
              (0.82, 0.28, 0.09),
              0, 40, col, "Fréquence en %",0,1, (11,7))
    if df[col].nunique()<6 and df[col].nunique()>2:
        pk.graph_circle(df[col], col, "Répartition des clients en fonction de la variable "+col)
    if df[col].nunique()<=2:
        pk.graph_pie(df[col], col, "Répartition des clients selon la variable "+col,["#f56315", "#f88e55"], (15,7)),
       

###############commentaires!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Il serait interessant de creer des regroupements ordinaux.

In [ ]:
reg_fisher_jenks(data, col, new_col, 5)

## Analyse bivariée

In [ ]:
gc.collect()

In [ ]:
for i in df.columns:
    print(i)

## correlation

In [ ]:
sns.set(rc={'figure.figsize':(10,4)})

data_corr = df.corr()

display(data_corr)
ax = sns.heatmap(data_corr, xticklabels = data_corr.columns , 
                 yticklabels = data_corr.columns, cmap = 'rocket_r')
plt.title("Matrice de corrélation")

plt.xlabel("Variables")

plt.ylabel("Variables")

## ACP

In [ ]:
df

In [ ]:
df_acp=df.copy()

In [ ]:
Z=pk.amulti_acp_standard(df_acp)

## Zoom segmentation RFM

### Recency

In [ ]:
df["order_purchase_timestamp_recency"].nunique()

In [ ]:
pk.graph_hist(df["order_purchase_timestamp_recency"],[0,50, 100, 200,300,400,500,600,772] ,"Distribution des clients en fonction de la variable order_purchase_timestamp_recency",
              "#f88e55", 0,772, 100, 0, 35000, "order_purchase_timestamp_recency", 'Fréquences',(11,7), 0, "")

### Frequency

# ########mettre en %

In [ ]:
df["order_id_size"].max()

In [ ]:
pk.graph_hist(df["order_id_size"],[0,1,2,3,4,5,6,7,8,9,10,15,17] ,"Distribution des clients en fonction de la variable order_id_size",
              "#f88e55", 0,17, 1, 0, 96000, "order_id_size", 'Fréquences',(11,7), 0, "")

### Revenue

In [ ]:
df["payment_value_sum_sum"].max()

In [ ]:
pk.graph_hist(df["payment_value_sum_sum"],[0,10,20,30,40,50,60,70,80,90, 100,500,1000,2000,13664.08] ,"Distribution des clients en fonction de la variable payment_value_sum_sum",
              "#f88e55", 0,13700, 10, 0, 55000, "payment_value_sum_sum", 'Fréquences',(11,7), 0, "")